In [ ]:
%pip install rdflib networkx fhirtordf --user

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dirlistproc: filename=dirlistproc-1.4.5-py3-none-any.whl size=6443 sha256=efecb1093d65452dd9cbd2938f8288817c9b0c5840fc376bce96fc87b176214c
  Stored in directory: c:\users\peter\appdata\local\pip\cache\wheels\02\5a\6f\688c77c1c42d1af11d9671e71b5cf1b4a1500551d21f4a86c5
Successfully built dirlistproc

   ------------- -------------------------- 2/6 [isodate]
  Attempting uninstall: rdflib
   ------------- -------------------------- 2/6 [isodate]
    Found existing installation: rdflib 7.2.1
   ------------- -------------------------- 2/6 [isodate]
   -------------------- ------------------- 3/6 [rdflib]
    Uninstalling rdflib-7.2.1:
   --------

In [61]:
from rdflib import Graph, URIRef, Literal, Namespace
from fhirtordf.loaders.fhirjsonloader import fhir_json_to_rdf
from fhirtordf.fhir.fhirmetavoc import FHIRMetaVoc
import networkx as nx
import json

In [67]:
g = fhir_json_to_rdf(
        json_fname='C:/Users/Peter/Documents/GitHub/daily-coding/fhir/Aaron697_Brekke496_2fa15bc7-8866-461a-9000-f739e425860a.json',
        # base_uri='https://peterdobbs77.github.io/fhir/',
        # metavoc = FHIRMetaVoc(mv_file_loc="http://build.fhir.org/fhir.ttl")
    )

In [68]:
print(g.serialize(format='turtle').decode('utf-8'))

@prefix fhir: <http://hl7.org/fhir/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<http://hl7.org/fhir/CarePlan/af160c6b-78e8-46d9-b49f-870fafe7f27e> a fhir:CarePlan ;
    fhir:nodeRole fhir:treeRoot .

<http://hl7.org/fhir/CarePlan/af160c6b-78e8-46d9-b49f-870fafe7f27e.ttl> a owl:Ontology ;
    owl:imports fhir:fhir.ttl .

<http://hl7.org/fhir/CarePlan/bdfcbba5-ee4f-4a8f-ac12-3da0301df4ef> a fhir:CarePlan ;
    fhir:nodeRole fhir:treeRoot .

<http://hl7.org/fhir/CarePlan/bdfcbba5-ee4f-4a8f-ac12-3da0301df4ef.ttl> a owl:Ontology ;
    owl:imports fhir:fhir.ttl .

<http://hl7.org/fhir/CareTeam/82697300-ce70-49e9-9012-79a33a776b59> a fhir:CareTeam ;
    fhir:nodeRole fhir:treeRoot .

<http://hl7.org/fhir/CareTeam/82697300-ce70-49e9-9012-79a33a776b59.ttl> a owl:Ontology ;
    owl:imports fhir:fhir.ttl .

<http://hl7.org/fhir/CareTeam/a542026f-5591-4bfc-903e-6216794179ce> a fhir:CareTeam ;
    fhir:nodeRole fhir:treeRoot .

<http://hl7.org/fhir/CareTeam/a542026f-5591-4bfc-903e-62167941

Something is off in the way that the references are being created. I think `fhirtordf` is struggling to establish references based on the structure of the synthea-generated data.

Instead let's try to do the conversion manually...

In [ ]:
with open('../../fhir/Aaron697_Brekke496_2fa15bc7-8866-461a-9000-f739e425860a.json', 'r') as f:
    fhir_bundle = json.load(f)

manual_graph = Graph()

# namespaces
FHIR = Namespace("http://hl7.org/fhir/")
EX = Namespace("http://example.org/")

for entry in fhir_bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    resource_id = URIRef(EX[resource_type + '/' + resource['id']])

    # add basic resource attributes
    manual_graph.add((resource_id, FHIR.resourceType, Literal(resource_type)))

    # traverse the attributes of the resource
    for key, value in resource.items():
        if key in ['encounter', 'subject', 'careteam', 'managingOrganization']:
            # build Reference to resource
            continue
        if key in ['coding']:
            # build Coding
            continue
        if key not in ['resourceType', 'id']:  # Skip resourceType and id
            if isinstance(value, dict):  # Handle nested objects
                for sub_key, sub_value in value.items():
                    if sub_key == 'reference':
                        manual_graph.add((resource_id, FHIR[key], Literal(sub_value)))
                    else:
                        manual_graph.add((resource_id, FHIR[key + '_' + sub_key], Literal(sub_value)))
            elif isinstance(value, list):  # Handle lists
                for item in value:
                    manual_graph.add((resource_id, FHIR[key], Literal(item)))
            else:
                manual_graph.add((resource_id, FHIR[key], Literal(value)))

# Serialize the graph in Turtle format
print(manual_graph.serialize(format='turtle').decode('utf-8'))

@prefix ns1: <http://hl7.org/fhir/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/CarePlan/af160c6b-78e8-46d9-b49f-870fafe7f27e> ns1:activity "{'detail': {'code': {'coding': [{'system': 'http://snomed.info/sct', 'code': '160670007', 'display': 'Diabetic diet'}], 'text': 'Diabetic diet'}, 'status': 'in-progress', 'location': {'display': 'PCP116891'}}}",
        "{'detail': {'code': {'coding': [{'system': 'http://snomed.info/sct', 'code': '229065009', 'display': 'Exercise therapy'}], 'text': 'Exercise therapy'}, 'status': 'in-progress', 'location': {'display': 'PCP116891'}}}" ;
    ns1:addresses "{'reference': 'urn:uuid:e471123e-f309-4862-b58b-2652cbe74e1a'}" ;
    ns1:careTeam "{'reference': 'urn:uuid:82697300-ce70-49e9-9012-79a33a776b59'}" ;
    ns1:category "{'coding': [{'system': 'http://snomed.info/sct', 'code': '698360004', 'display': 'Diabetes self management plan'}], 'text': 'Diabetes self management plan'}" ;
    ns1:encounter "urn:uuid:6932cd4f-fa1a-